In [1]:
%pylab notebook

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pandas import *

In [24]:
frm = read_csv('./tabula-AD9174.csv')
frm

,Addr.,Name,Bits,Bit Name,Settings,Description,Reset,Access
0,0x000,SPI_INTFCONFA,7,SOFTRESET_M,NaN,Soft reset (mirror). Set this bit to mirror Bi...,0x0,R
1,NaN,NaN,6,LSBFIRST_M,NaN,LSB first (mirror). Set this bit to mirror Bit 1.,0x0,R
2,NaN,NaN,5,ADDRINC_M,NaN,Address increment (mirror). Set this bit to,0x0,R
3,NaN,NaN,NaN,NaN,NaN,mirror Bit 2.,NaN,NaN
4,NaN,NaN,4,SDOACTIVE_M,NaN,SDO active (mirror). Set this bit to mirror Bi...,0x0,R
5,NaN,NaN,3,SDOACTIVE,NaN,SDO active. Enables 4-wire SPI bus mode.,0x0,R/W
6,NaN,NaN,2,ADDRINC,NaN,"Address increment. When set, this bit",0x0,R/W
7,NaN,NaN,NaN,NaN,NaN,causes incrementing streaming addresses;,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,"otherwise, descending addresses are",NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,generated.,NaN,NaN


In [114]:
from collections import defaultdict
from json import dump

In [88]:
def isStr(x):
    if type(x) is str and len(x) > 0:
        return x
    return None

def isInt(x):
    try:
        val = int(x, 16)
    except:
        val = None
    return val

In [113]:
regs = dict()
for i, f in frm.iterrows():
    if f['Addr.'] == 'Addr.':
        continue
        
    # Setup a new empty register entry if an address is found
    x = isInt(f['Addr.'])
    if x is not None:
        addr = x
        regs[addr] = {
            'name': '', 
            'bits': defaultdict(lambda: {'name': '', 'reset': None, 'description': ''})
        }
        
    x = isStr(f['Name'])
    if x is not None:
        regs[addr]['name'] += x

    x = isStr(f['Bits'])
    if x is not None:
        bit_key = x
       
    x = isStr(f['Bit Name'])
    if x is not None:    
        regs[addr]['bits'][bit_key]['name'] += x
        
    x = isInt(f['Reset'])
    if x is not None:
        regs[addr]['bits'][bit_key]['reset'] = x
        
    x = isStr(f['Access'])
    if x is not None:    
        regs[addr]['bits'][bit_key]['access'] = x
        
    x = isStr(f['Description'])
    y = isStr(f['Settings'])
    if y is not None:
        x = y + ': ' + x
    if x is not None:
        if len(regs[addr]['bits'][bit_key]['description']) > 0:
            x = ' ' + x
        regs[addr]['bits'][bit_key]['description'] += x

In [118]:
with open('regs_ad9174.json', 'w') as f:
    dump(regs, f, indent=4)